In [2]:
import kfp
from kfp.components import load_component_from_file
import kfp.dsl as dsl

train_op = load_component_from_file("components/train/component.yaml")

@dsl.pipeline(
    name="KWS-train-test-pipe"
)
def pipeline(
    yaml_file:str,
    env_file:str, 
    data_path:str, 
    bucket_name:str,
    remote_dir:str ):
    
    train_task = train_op(
        yaml_file,
        env_file, 
        data_path, 
        bucket_name,
        remote_dir)

kfp.compiler.Compiler(mode=kfp.dsl.PipelineExecutionMode.V2_COMPATIBLE).compile(
    pipeline_func=pipeline,
    package_path='pipeline.yaml')


/home/lebugcat/miniconda3/envs/kws-dev-env/lib/python3.9/site-packages/kfp/compiler/compiler.py:79: UserWarning: V2_COMPATIBLE execution mode is at Beta quality. Some pipeline features may not work as expected.
  warnings.warn('V2_COMPATIBLE execution mode is at Beta quality.'


ValueError: @dsl.pipeline decorator name field is required in v2 compatible mode